# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [1]:
from scipy import stats
dir(stats)

['__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_binned_statistic',
 '_constants',
 '_continuous_distns',
 '_discrete_distns',
 '_distn_infrastructure',
 '_distr_params',
 '_multivariate',
 '_stats',
 '_stats_mstats_common',
 '_tukeylambda_stats',
 'absolute_import',
 'alpha',
 'anderson',
 'anderson_ksamp',
 'anglit',
 'ansari',
 'arcsine',
 'argus',
 'bartlett',
 'bayes_mvs',
 'bernoulli',
 'beta',
 'betaprime',
 'binned_statistic',
 'binned_statistic_2d',
 'binned_statistic_dd',
 'binom',
 'binom_test',
 'boltzmann',
 'boxcox',
 'boxcox_llf',
 'boxcox_normmax',
 'boxcox_normplot',
 'bradford',
 'burr',
 'burr12',
 'cauchy',
 'chi',
 'chi2',
 'chi2_contingency',
 'chisquare',
 'circmean',
 'circstd',
 'circvar',
 'combine_pvalues',
 'contingency',
 'cosine',
 'crystalball',
 'cumfreq',
 'describe',
 'dgamma',
 'dirichlet',
 'distributions',
 'division',
 'dlaplace',
 'dweibull',
 'energy_dist

In [2]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [3]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [4]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test. We will also dig a bit deeper into how the test statistic is calculated based on the sample error, and visually what it looks like to have 1 or 2 "tailed" t-tests.

In [0]:
import random
random.seed(10)
import scipy
import numpy as np
import pandas as pd

In [0]:
def variance(nums):
  n = len(nums)
  return np.sum((nums - scipy.mean(nums))**2)/(n - 1)
  
def stdev(nums):
  return variance(nums)**(1/2)

  

In [0]:
population = []

for _ in range(1000):
  population.append(random.randint(0,1))
  
sample = random.sample(population, 100)

In [8]:
df = pd.DataFrame({'likes_coke': sample})
df.head()

df.likes_coke.mean()

0.46

In [9]:
#t = variance between groups / variance within groups

n = len(df.likes_coke)
t_stat = (scipy.mean(df.likes_coke) - scipy.mean(population))/(stdev(df.likes_coke)/n**(1/2))
t_stat

-0.9382955561680112

In [10]:
scipy.stats.ttest_1samp(df['likes_coke'], scipy.mean(population))

Ttest_1sampResult(statistic=-0.9382955561680112, pvalue=0.3503760732929039)

In [11]:
samp2 = random.sample(population, 3)
samp2

[1, 0, 0]

In [12]:
scipy.stats.ttest_1samp(samp2, scipy.mean(population))

Ttest_1sampResult(statistic=-0.521, pvalue=0.6543098596040877)

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
#1) Load and clean the data:


url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

cols = ['party', 'Handicapped Infants', 'Water Project Cost Sharing', 'Budget Resolution', 
        'Physician Fee Freeze', 'El Salvador Aid', 'Religious Groups in Schools', 
        'Anti satelite Test Ban', 'Aid to Nicaraguan Contras', 'Mx-Missile', 'Immigration', 
        'Synfuels Corporation Cutback', 'Education Spending', 'Superfund Right to Sue',
        'Crime', 'Duty-Free Exports', 'Export Administration Act S Africa']

data = pd.read_csv(url, header=None, names=cols, na_values='?')
data = data.replace({'y':1,'n':0})

In [0]:
#split into 2 dataframes by party affiliation:

rep = data[data['party'] == 'republican']

dem = data[data['party'] == 'democrat']

In [15]:
'''Perform ttest_ind calculations for each issue by looping thru the columns and
recording the tstat, the p value, and then a boolean whether the pvalue is less 
than 0.01 and also whether it is greater than 0.1.'''

ttest_results = []

for col in cols[1:]:
  t, p = stats.ttest_ind(dem[col].dropna(), rep[col].dropna())
  ttest_results.append([col, round(t, 3), p < 0.01, p > 0.1])

ttest_results

[['Handicapped Infants', 9.205, True, False],
 ['Water Project Cost Sharing', -0.089, False, True],
 ['Budget Resolution', 23.213, True, False],
 ['Physician Fee Freeze', -49.367, True, False],
 ['El Salvador Aid', -21.137, True, False],
 ['Religious Groups in Schools', -9.738, True, False],
 ['Anti satelite Test Ban', 12.526, True, False],
 ['Aid to Nicaraguan Contras', 18.052, True, False],
 ['Mx-Missile', 16.438, True, False],
 ['Immigration', -1.736, False, False],
 ['Synfuels Corporation Cutback', 8.294, True, False],
 ['Education Spending', -20.501, True, False],
 ['Superfund Right to Sue', -13.511, True, False],
 ['Crime', -16.342, True, False],
 ['Duty-Free Exports', 12.853, True, False],
 ['Export Administration Act S Africa', 6.849, True, False]]

In [0]:
#convert the results into a pandas dataframe and edit the column names

results_df = pd.DataFrame(ttest_results)
results_df = results_df.rename(columns = {0:'issue', 1:'t-stat', 2:'p < .01', 3:'p > 0.1'})

In [17]:
#Add a column to demonstrate which party was more likely to vote Yes for a given issue:

results_df['More Likely to Support'] = results_df['t-stat'] > 0

results_df['More Likely to Support'] = results_df['More Likely to Support'].replace({True:'Democrats', False: 'Republicans'})

results_df

,issue,t-stat,p < .01,p > 0.1,More Likely to Support
0,Handicapped Infants,9.205,True,False,Democrats
1,Water Project Cost Sharing,-0.089,False,True,Republicans
2,Budget Resolution,23.213,True,False,Democrats
3,Physician Fee Freeze,-49.367,True,False,Republicans
4,El Salvador Aid,-21.137,True,False,Republicans
5,Religious Groups in Schools,-9.738,True,False,Republicans
6,Anti satelite Test Ban,12.526,True,False,Democrats
7,Aid to Nicaraguan Contras,18.052,True,False,Democrats
8,Mx-Missile,16.438,True,False,Democrats
9,Immigration,-1.736,False,False,Republicans


In [18]:
#2) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
#statistically significant Democrat issues:

condition = (results_df['t-stat'] > 0) & (results_df['p < .01'] == True)
dem_issues = results_df[condition]['issue']

dem_issues

0                    Handicapped Infants
2                      Budget Resolution
6                 Anti satelite Test Ban
7              Aid to Nicaraguan Contras
8                             Mx-Missile
10          Synfuels Corporation Cutback
14                     Duty-Free Exports
15    Export Administration Act S Africa
Name: issue, dtype: object

In [19]:
#3) Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
#statistically significant Republican issues:

condition = (results_df['t-stat'] < 0) & (results_df['p < .01'] == True)
rep_issues = results_df[condition]['issue']

rep_issues

3            Physician Fee Freeze
4                 El Salvador Aid
5     Religious Groups in Schools
11             Education Spending
12         Superfund Right to Sue
13                          Crime
Name: issue, dtype: object

In [20]:
#4) Using hypothesis testing, find an issue where the difference between republicans 
#   and democrats has p > 0.1 (i.e. there may not be much of a difference)

insignificant_issues = results_df[results_df['p > 0.1'] == True]['issue']
insignificant_issues

1    Water Project Cost Sharing
Name: issue, dtype: object

# Resources

- https://homepage.divms.uiowa.edu/~mbognar/applets/t.html
- https://rpsychologist.com/d3/tdist/
- https://gallery.shinyapps.io/tdist/
- https://en.wikipedia.org/wiki/Standard_deviation#Sample_standard_deviation_of_metabolic_rate_of_northern_fulmars